In [373]:
def mrr(data):
    mrr_metrics = []

    log_question_counter = 0
    log_checker_1 = 0
    log_checker_2 = 0
    log_checker_3 = 0

    questions = data['question'].to_list()
    for question in questions:
        relevant_docs = get_rel_chunks(question)

        pos = 1
        for doc in relevant_docs:
            a = data.loc[data['question']==question, 'answer'].values[0]
            if a in doc:
                mrr_metrics.append(1/pos)
                # print(f'CHECK1-полное совпадение подстроки в чанке\n\nПравильный ответ\n\n{a}\n\nЧанк:\n\n{doc}')
                log_checker_1 += 1
                break
            elif longest_common_substring(a, doc) >= len(a.split(" ")) // 4:
                mrr_metrics.append(1/pos)
                print(f'CHECK2-пересечение\n\nПравильный ответ: {a}\nЧанк:\n {doc}\n\nПересечение: {longest_common_substring(a, doc)}\n\n')
                log_checker_2 += 1
                break
            else:
                # print(f'CHECK3-нет пересечения и подстроки\n\nОтвет:\n\n{a}\n\nЧанк:\n\n {doc}')
                log_checker_3 += 1
            pos += 1
        
        log_question_counter += 1

    print(f'Q:{log_question_counter}')
    print(f'C1:{log_checker_1}')
    print(f'C2:{log_checker_2}')
    print(f'C3:{log_checker_3}')

    while len(mrr_metrics) < len(questions):
        mrr_metrics.append(0)
    return f'Mean MRR = {sum(mrr_metrics)/len(mrr_metrics)}'

In [374]:
mrr(df_texts)


CHECK2-пересечение

Правильный ответ: В ноябре 2022 г. губернатор Чукотки Роман Копин сообщал, что грузооборот порта Певек за 2022 г. увеличился на 35% и достиг 380 000 т.
Чанк:
 век за 2022 г. увеличился на 35% и достиг 380 000 т. Порт принял 60 судов, на 17 больше, чем в 2021 г. Навигация в Певеке длится с 3 июля до 25 октября. Порт оснащен причалом длиной 500 м. Из распоряжения правительства, подписанного Мишустиным 1 марта 2023 г., следует, что общий объем инвестиций в инфраструктуру составит 55,21 млрд руб. Оставшиеся 27,58 млрд руб. будут направлены в проект из внебюджетных источников. Бюджетные средства разделены на две части: 17,26 млрд руб. будет направлено на строительство грузовых причалов, оставшиеся 10,37 млрд руб. – на инфраструктуру под модернизированный энергоблок плавучей атомной теплоэлектростанции (ПАТЭС, принадлежит «Росатому»), который обеспечит энергией разработку Баимского месторождения. Баимское – одна из крупнейших в мире неосвоенных медных площадей. Месторожде

'Mean MRR = 0.8974358974358975'

In [ ]:
# q = 'Как изменились реальные распологаемые доходы россиян в первом квартале 2020 года?'
# mrr_data.loc[mrr_data['question']==q, 'answer'][0]

mrr(mrr_data)

In [ ]:
def longest_common_substring(str1, str2):
    # Создаем таблицу для динамического программирования
    table = [[0] * (len(str2) + 1) for _ in range(len(str1) + 1)]
    longest = 0
    lcs = ""

    # Заполнение таблицы и поиск наибольшей длины LCS
    for i in range(1, len(str1) + 1):
        for j in range(1, len(str2) + 1):
            if str1[i - 1] == str2[j - 1]:
                table[i][j] = table[i - 1][j - 1] + 1
                if table[i][j] > longest:
                    longest = table[i][j]
                    lcs = str1[i - longest:i]
            else:
                table[i][j] = 0
    return len(lcs.split(' '))


# Пример использования функции
str1 = "Hello there, how are you?"
str2 = "I am here, how about you?"
common_substring = longest_common_substring(a, b)
print(f"Наибольшая общая подстрока: '{common_substring}'")


In [ ]:
def mean_bert_score(data):
    bert_score_metrics = []

    questions = data['question'].to_list()
    for question in questions:
        relevant_docs = get_rel_chunks(question)

        for doc in relevant_docs:
            

In [ ]:
import numpy as np
np.mean([1,23,34,255])

In [ ]:
%%capture
!pip install sentence_transformers chromadb langchain llama_index llama-index-vector-stores-chroma llama-index-embeddings-huggingface llama_index.finetuning pydantic
# !pip install llama_index.evaluation

In [ ]:
# import chromadb
# from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
# embedding_function = SentenceTransformerEmbeddingFunction(model_name='intfloat/multilingual-e5-large')

In [ ]:
# Сохраняем текста
import pandas as pd

df_texts = pd.read_csv('data/test_data_with_questions_answ.csv')

texts = "\n\n".join(df_texts['texts']) # объединяем все текста

In [370]:
# сплитим текст на чанки по токенам

from langchain.text_splitter import SentenceTransformersTokenTextSplitter

def text_token_splitter(texts, model_name, tokens_per_chunk):
    # token_splitter = SentenceTransformersTokenTextSplitter(model_name=model_name, chunk_overlap=100, tokens_per_chunk=tokens_per_chunk)
    token_splitter = SentenceTransformersTokenTextSplitter(model_name=model_name, chunk_overlap=0, tokens_per_chunk=tokens_per_chunk)
    return token_splitter.split_text(texts)

texts = "\n\n".join(df_texts['texts']) # объединяем все текста
model_name = 'intfloat/multilingual-e5-large'
tokens_per_chunk = 510
token_spliter_chunks = text_token_splitter(texts, model_name, tokens_per_chunk)

print(f"\nTotal chunks: {len(token_spliter_chunks)}")


Total chunks: 138


In [ ]:
# FOR TEST ONLY
def check_tokens(text, model_name):
    tokenizer = SentenceTransformersTokenTextSplitter(model_name=model_name).tokenizer
    encoded_input = tokenizer(text, return_tensors='pt')
    return tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0])

check_tokens('Тестовый текст', 'intfloat/multilingual-e5-large')

In [ ]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
embedding_function = SentenceTransformerEmbeddingFunction(model_name=model_name)


In [ ]:
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("test", embedding_function=embedding_function)

ids = [str(i) for i in range(len(token_spliter_chunks))]

chroma_collection.add(ids=ids, documents=token_spliter_chunks)
chroma_collection.count()

In [ ]:
query = ques

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]
output_lst = []

for document in retrieved_documents:
    output_lst.append(document)
    
output_lst

In [ ]:
document

In [ ]:
val = df_texts.sample(1)

In [ ]:
ques = val.question.iloc[0]

In [ ]:
tx = val.texts.iloc[0]

In [ ]:
answ = val.answer.iloc[0]

In [ ]:
# chroma_client = chromadb.EphemeralClient()
# chroma_collection = chroma_client.create_collection("quickstart")

In [ ]:
# from llama_index.core import VectorStoreIndex, StorageContext
# from llama_index.vector_stores.chroma import ChromaVectorStore

In [ ]:
!pip install llama_index

In [ ]:
%%capture
# !pip install llama-index-vector-stores-chroma
!pip install llama-index-embeddings-huggingface

In [ ]:
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

Settings.embed_model = HuggingFaceEmbedding(
    model_name="intfloat/multilingual-e5-large"
)

vector_store = ChromaVectorStore(chroma_collection=chroma_collection)


In [ ]:
vector_store

In [ ]:
from llama_index.core import VectorStoreIndex, StorageContext

index = VectorStoreIndex.from_vector_store(vector_store)
vector_retriever = index.as_retriever(similarity_top_k=5)

In [ ]:
index = 0
counter = len(df_texts) - 1
df_texts.loc[index, 'question']
df_texts.loc[index, 'answer']



In [ ]:
# Create inputs for EmbeddingQAFinetuneDataset
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

counter = len(df_texts) - 1

queries_dict = {f"query_{index}": df_texts.loc[index, 'question'] for index in range(counter)}
corpus_dict = {f"corpus_{index}": df_texts.loc[index, 'answer'] for index in range(counter)}
relevant_docs_dict = {f"query_{index}":[f"corpus_{index}"] for index in range(counter)}

# Create QA dataset
qa_dataset = EmbeddingQAFinetuneDataset(
    queries=queries_dict,
    corpus=corpus_dict,
    relevant_docs=relevant_docs_dict
)

In [ ]:
qa_dataset

In [ ]:
from llama_index.core.evaluation import RetrieverEvaluator


retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=vector_retriever
)
eval_results = await retriever_evaluator.aevaluate_dataset(qa_dataset)

In [ ]:
for idx in range(len(eval_results)):
    print(eval_results[idx].metric_dict)

In [ ]:
# queries_dict = {
#     "id" : "rel_doc"
# }

In [ ]:
# queries_dict = {f"query_{index}":filtered_queries[index] for index in range(counter)}
# corpus_dict = {f"corpus_{index}":corpus[index] for index in range(counter)}
# relevant_docs_dict = {f"query_{index}":[f"corpus_{index}"] for index in range(counter)}

# # Create QA dataset
# qa_dataset = EmbeddingQAFinetuneDataset(
#     queries=queries_dict,
#     corpus=corpus_dict,
#     relevant_docs=relevant_docs_dict
# )

# wfgwgfwfwfwfw

In [ ]:
import evaluate

predictions = ['Лиза Янковская', 'Лиза Янковская', 'Лиза Янковская', 'Лиза Янковская', 'Лиза Янковская'] #
references = output_lst2
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
print(results)

In [ ]:
from evaluate import load
bertscore = load("bertscore")
predictions = output_lst2  #
references = ['Лиза Янковская.', 'Лиза Янковская.', 'Лиза Янковская.', 'Лиза Янковская.', 'Лиза Янковская.']
results = bertscore.compute(predictions=predictions, references=references, lang="ru")
results

In [ ]:
output_lst2

In [ ]:
from evaluate import load
bertscore = load("bertscore")
predictions = output_lst2  #
references = [asa, asa, asa, asa, asa]
results = bertscore.compute(predictions=predictions, references=references, lang="ru")
results

In [ ]:
query = 'Как изменились реальные распологаемые доходы россиян в первом квартале 2020 года?'

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]
output_lst2 = []

for document in retrieved_documents:
    output_lst2.append(document)
    
output_lst2

In [ ]:
output_lst2[0] = 'Реальные располагаемые доходы россиян уже начали сокращаться Пока это эффект девальвации рубля, а не режима самоизоляции'

In [ ]:
output_lst2[0] = 'Реальные располагаемые доходы россиян уже начали сокращаться Пока это эффект девальвации рубля, а не режима самоизоляции За I квартал 2020 г. реальные располагаемые денежные доходы россиян (за вычетом обязательных платежей) сократились на 0,2%'

In [ ]:
asa = df_texts_copy.iloc[0].answer
asa

In [ ]:
t = df_texts_copy.texts.tolist()
t = [[t] for t in t]
t

In [ ]:
def calculate_mrr(ground_truths, predictions):
    """
    Calculate the Mean Reciprocal Rank (MRR).
    
    Parameters:
    - ground_truths (list of list): Each inner list contains the ground truth items for a query.
    - predictions (list of list): Each inner list contains the predicted items for a query, ordered by relevance.

    Returns:
    - float: The MRR score.
    """
    rr_sum = 0.0
    for gt, preds in zip(ground_truths, predictions):
        for rank, pred in enumerate(preds, start=1):
            if pred in gt:
                rr_sum += 1.0 / rank
                break
    mrr = rr_sum / len(ground_truths)
    return mrr


ground_truths = [ans, ans, ans, ans, ans]
predictions = output_lst

In [ ]:
answrs_list = df_texts.answer.tolist()

In [ ]:
df_texts_copy = df_texts.copy()

In [ ]:
df_texts_copy.iloc[0].answer

In [ ]:
calculate_mrr(ground_truths, predictions)

In [ ]:
chunks = token_spliter_chunks.copy()
answers = answrs_list.copy()

print(f'Чанков - {len(chunks)}, ответов - {len(answers)}')

In [ ]:
mrr_data = df_texts[df_texts['answer'].isin(w_a)]
mrr_data.shape

In [ ]:
w_a

In [ ]:
len(list(set(w_a)))


In [ ]:
counter = 0
w_a = []
for i in range(len(answers)):
    answer = answers.pop()
    for j in range(len(chunks)):
        if answer.lower() in chunks[j].lower():
            counter += 1
            w_a.append(answer)
            # print(f'{answer}\n\n{chunk}\n\n{"=="*20}')
            # answers.remove(answer)
    

            
print(f'Чанков - {len(chunks)}, ответов - {len(answers)}')
print(counter)

In [ ]:
def get_rel_chunks(query):
    results = chroma_collection.query(query_texts=[query], n_results=5)
    retrieved_documents = results['documents'][0]
    output_lst = []

    for document in retrieved_documents:
        output_lst.append(document)
        
    return output_lst 

In [304]:
a = 'За I квартал 2020 г. реальные располагаемые денежные доходы россиян (за вычетом обязательных платежей) сократились на 0,2%'

In [306]:
b = 'денежные доходы россиян (за вычетом обязательных платежей) сократились на 0,2% в годовом выражении, говорится в опубликованных Росстатом данных. Ограничения из-за коронавируса, которые власти начали вводить во второй половине марта, еще не сказались на доходах людей, а основной удар по ним произойдет во II квартале, предупреждают экономисты. Главный фактор снижения доходов – девальвация рубля после развала сделки ОПЕК+, говорит главный экономист BCS Global Markets Владимир Тихомиров. Впервые с февраля 2016 г. курс рубля опускался ниже 80 руб./$. Режим самоизоляции был введен только в последние дни марта и вряд ли уже сказался на статистике, говорит он. Хотя в марте уже начали сокращаться зарплаты, на которые приходится основная доля доходов'

In [320]:
token_spliter_chunks[0]

'Реальные располагаемые доходы россиян уже начали сокращаться Пока это эффект девальвации рубля, а не режима самоизоляции За I квартал 2020 г. реальные располагаемые денежные доходы россиян (за вычетом обязательных платежей) сократились на 0,2% в годовом выражении, говорится в опубликованных Росстатом данных. Ограничения из-за коронавируса, которые власти начали вводить во второй половине марта, еще не сказались на доходах людей, а основной удар по ним произойдет во II квартале, предупреждают экономисты. Главный фактор снижения доходов – девальвация рубля после развала сделки ОПЕК+, говорит главный экономист BCS Global Markets Владимир Тихомиров. Впервые с февраля 2016 г. курс рубля опускался ниже 80 руб./$. Режим самоизоляции был введен только в последние дни марта и вряд ли уже сказался на статистике, говорит он. Хотя в марте уже начали сокращаться зарплаты, на которые приходится основная доля доходов (более 60% за I квартал 2020 г.), предупреждает Тихомиров. Компании, опрошенные Цен

In [321]:
token_spliter_chunks[1][:200]

'года начиная с 2014 г., а за 2018 г. выросли на скромные 0,2%. 16% россиян уже столкнулись с серьезным падением доходов Люди ждут, что ситуация будет ухудшаться, показал опрос «ИнФОМа» Падение доходов'